In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11 cuml-cu11
!pip install pandas==1.5.3

In [ ]:
!pip install \
    cuml-cu12 --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12 --extra-index-url=https://pypi.nvidia.com \
    rmm-cu12 --extra-index-url=https://pypi.nvidia.com \
    rapids-dask-dependency --extra-index-url=https://pypi.nvidia.com

print("Cài đặt lại RAPIDS cho CUDA 12 hoàn tất!")

In [ ]:
#test cross:
#cross
import os
import numpy as np
import warnings
import pickle
import cuml
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import StratifiedKFold
import cudf
import cupy as cp
from cuml.svm import SVC as CuSVC
from cuml.metrics import accuracy_score, confusion_matrix
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler
try:
    #from cuml.metrics import accuracy_score, classification_report
    print("CuML imported successfully - GPU support enabled!")
    GPU_AVAILABLE = True
except ImportError:
    print("CuML not available, falling back to CPU sklearn")
    exit(0)
    from sklearn.svm import SVC as CuSVC
    GPU_AVAILABLE = False

print("Checking GPU availability...")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader,nounits

def main(folder_path):
    svm_params = {'C': 1.0, 'kernel': 'rbf', 'gamma': 'scale', 'probability': True}
    with open("log_result_SVM_cross_validation.txt", "a") as log_file:
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".csv"):
                file_path = os.path.join(folder_path, file_name)
                print(f"Processing file: {file_name}")

                cols = ["Destination Port", "Flow Duration", "Total Fwd Packets", "Total Backward Packets",
                        "Total Length of Fwd Packets", "Total Length of Bwd Packets", "Fwd Packet Length Max",
                        "Fwd Packet Length Min", "Fwd Packet Length Mean", "Fwd Packet Length Std",
                        "Bwd Packet Length Max", "Bwd Packet Length Min", "Bwd Packet Length Mean",
                        "Bwd Packet Length Std", "Flow Bytes/s", "Flow Packets/s", "Flow IAT Mean",
                        "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Mean",
                        "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean",
                        "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags",
                        "Fwd URG Flags", "Bwd URG Flags", "Fwd Header Length", "Bwd Header Length",
                        "Fwd Packets/s", "Bwd Packets/s", "Min Packet Length", "Max Packet Length",
                        "Packet Length Mean", "Packet Length Std", "Packet Length Variance", "FIN Flag Count",
                        "SYN Flag Count", "RST Flag Count", "PSH Flag Count", "ACK Flag Count", "URG Flag Count",
                        "CWE Flag Count", "ECE Flag Count", "Down/Up Ratio", "Average Packet Size",
                        "Avg Fwd Segment Size", "Avg Bwd Segment Size", "Fwd Header Length.1", "Fwd Avg Bytes/Bulk",
                        "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk",
                        "Bwd Avg Bulk Rate", "Subflow Fwd Packets", "Subflow Fwd Bytes", "Subflow Bwd Packets",
                        "Subflow Bwd Bytes", "Init_Win_bytes_forward", "Init_Win_bytes_backward",
                        "act_data_pkt_fwd", "min_seg_size_forward", "Active Mean", "Active Std",
                        "Active Max", "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min", "Label"]

                df = cudf.read_csv(file_path, names=cols)
                df["Label"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)
                print(df["Label"].value_counts())
                print(df.head())

                # Preprocess data
                for col in df.columns[:-1]:
                    df[col] = cudf.to_numeric(df[col], errors='coerce')
                df.dropna(inplace=True)

                X = df[df.columns[:-1]].astype(cp.float32)
                y = df[df.columns[-1]].astype(cp.int32)
                X = X.fillna(0.0)
                X = X.replace([float('inf'), -float('inf')], 0.0)

                scaler = StandardScaler()
                X = scaler.fit_transform(X)

                print("Starting 5-fold cross-validation on GPU...")
                cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

                fold_f1_scores = []
                y_cpu_for_split = y.to_numpy()

                for fold, (train_idx, val_idx) in enumerate(cv.split(X, y_cpu_for_split)):
                    print(f"--- Fold {fold + 1}/5 ---")

                    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

                    model = CuSVC(**svm_params)
                    model.fit(X_train, y_train)
                    y_pred_fold = model.predict(X_val)

                    cm = confusion_matrix(y_val, y_pred_fold)
                    tn, fp, fn, tp = cm.ravel()
                    precision = tp / (tp + fp + 1e-6)
                    recall = tp / (tp + fn + 1e-6)
                    f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
                    score = f1

                    fold_f1_scores.append(float(score))
                    print(f"Fold {fold + 1} F1 Score: {score:.4f}")

                mean_f1_cv = np.mean(fold_f1_scores)
                std_f1_cv = np.std(fold_f1_scores)

                print("Training final model on all data for reporting...")
                final_model = CuSVC(**svm_params)
                final_model.fit(X, y)
                y_pred_all = final_model.predict(X)

                y_cpu_all = y.to_numpy()
                y_pred_cpu_all = y_pred_all.to_numpy()


                log_file.write(f"Result of file: {file_name}\n")
                log_file.write(f"Classification Report: \n{classification_report(y_cpu_all, y_pred_cpu_all)}")
                log_file.write(f"Accuracy: {accuracy_score(y, y_pred_all)*100:.2f}%\n")
                log_file.write(f"Params: {svm_params}\n")
                log_file.write(f"Mean F1 Score (5-fold CV): {mean_f1_cv:.4f} (+/- {std_f1_cv * 2:.4f})\n")
                log_file.write("Model training complete.\n")
                log_file.write("---------------------------------------------------\n\n")


                print(f"Result of file: {file_name}\n")
                print(f"Classification Report: \n{classification_report(y_cpu_all, y_pred_cpu_all)}")
                print(f"Accuracy: {accuracy_score(y, y_pred_all)*100:.2f}%\n")
                print(f"Params: {svm_params}\n")
                print(f"Mean F1 Score (5-fold CV): {mean_f1_cv:.4f} (+/- {std_f1_cv * 2:.4f})\n")
                print("Model training complete.\n")
                print("---------------------------------------------------\n\n")

if __name__ == "__main__":
    folder_path = "/content/"
    main(folder_path)

CuML imported successfully - GPU support enabled!
Checking GPU availability...
Tesla T4, 15360, 15092
Processing file: merged_dataset.csv
Label
0    2273097
1     557647
Name: count, dtype: int64
    Destination Port   Flow Duration   Total Fwd Packets  \
0   Destination Port   Flow Duration   Total Fwd Packets   
1              54865               3                   2   
2              55054             109                   1   
3              55055              52                   1   
4              46236              34                   1   

    Total Backward Packets  Total Length of Fwd Packets  \
0   Total Backward Packets  Total Length of Fwd Packets   
1                        0                           12   
2                        1                            6   
3                        1                            6   
4                        1                            6   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0   Total Length of Bwd Packe

/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


--- Fold 1/5 ---
Fold 1 F1 Score: 0.9265
--- Fold 2/5 ---
Fold 2 F1 Score: 0.9264
--- Fold 3/5 ---
Fold 3 F1 Score: 0.9274
--- Fold 4/5 ---
Fold 4 F1 Score: 0.9267
--- Fold 5/5 ---
Fold 5 F1 Score: 0.9266
Training final model on all data for reporting...
Result of file: merged_dataset.csv

Classification Report: 
              precision    recall  f1-score   support

           0       0.98      0.98      0.98   2272688
           1       0.92      0.93      0.93    556697

    accuracy                           0.97   2829385
   macro avg       0.95      0.96      0.95   2829385
weighted avg       0.97      0.97      0.97   2829385

Accuracy: 97.14%

Params: {'C': 1.0, 'kernel': 'rbf', 'gamma': 'scale', 'probability': True}

Mean F1 Score (5-fold CV): 0.9267 (+/- 0.0007)

Model training complete.

---------------------------------------------------


